In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
import csv


from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel
import re
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import nltk
nltk.download('stopwords')
nltk.download('punkt')
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [ ]:
df = pd.read_csv("dataset-ikan.csv")

In [ ]:
df

,Title,Ingredients,Steps,Loves,URL
0,Gurame Saus Padang,Bahan utama:--1 ekor gurame--Bumbu untuk saus:...,Cuci bersih ikan gurame yang akan dimasak. Set...,2,/id/resep/4474009-gurame-saus-padang
1,Ikan Kembung Bakar Teflon,"1/2 kg ikan kembung sate, bersihkan--1 buah je...","Kucuri ikan dengan jeruk, diamkan 5 menit--Lum...",2,/id/resep/4474225-ikan-kembung-bakar-teflon
2,Mujaer asam pedas manis,1/2 kg ikan mujaer (stok gurame habis)--2 buah...,Bersihkan ikan sampai benar-benar bersih. Kera...,1,/id/resep/4473689-mujaer-asam-pedas-manis
3,Gurame Asam manis,1/2 kg ikan gurame--2 buah wortel (potong sepe...,Uleg kunir sama garam kasih air lalu guraminya...,5,/id/resep/4473428-gurame-asam-manis
4,Asam padeh ikan tuna,1/2 kg ikan tuna--6 siung bawang merah--4 siun...,"Haluskan bumbu (cabai merah, kemiri, bawang, k...",7,/id/resep/4473138-asam-padeh-ikan-tuna
...,...,...,...,...,...
1927,Mie rebus tongkol suwir ala neng tika,1 bungkus mie rebus (kari ayam)--2 potong ikan...,Tumis bumbu halus hingga harum masukan tongkol...,0,/id/resep/4003324-mie-rebus-tongkol-suwir-ala-...
1928,Tongkol pecak sambel korek ala neng tika,1 ekor ikan tongkol (potong kecil cuci bersih)...,Goreng ikan tongkol hingga kering lalu goreng ...,3,/id/resep/4001241-tongkol-pecak-sambel-korek-a...
1929,Lele frozen berbumbu,1 kg lele isi 10--Bumbu rendam:--1 sdt garam--...,"Bersih kan lele buang ingsang nya, kerat2 cuci...",5,/id/resep/4002895-lele-frozen-berbumbu
1930,Gurami saus padang,Gurami--1 buah Jeruk nipis--Garam--Lada--3 siu...,"Bersihkan ikan gurami, buang isi perut cuci be...",4,/id/resep/4002591-gurami-saus-padang


In [ ]:
df.dropna(inplace=True)
df.head()

,Title,Ingredients,Steps,Loves,URL
0,Gurame Saus Padang,Bahan utama:--1 ekor gurame--Bumbu untuk saus:...,Cuci bersih ikan gurame yang akan dimasak. Set...,2,/id/resep/4474009-gurame-saus-padang
1,Ikan Kembung Bakar Teflon,"1/2 kg ikan kembung sate, bersihkan--1 buah je...","Kucuri ikan dengan jeruk, diamkan 5 menit--Lum...",2,/id/resep/4474225-ikan-kembung-bakar-teflon
2,Mujaer asam pedas manis,1/2 kg ikan mujaer (stok gurame habis)--2 buah...,Bersihkan ikan sampai benar-benar bersih. Kera...,1,/id/resep/4473689-mujaer-asam-pedas-manis
3,Gurame Asam manis,1/2 kg ikan gurame--2 buah wortel (potong sepe...,Uleg kunir sama garam kasih air lalu guraminya...,5,/id/resep/4473428-gurame-asam-manis
4,Asam padeh ikan tuna,1/2 kg ikan tuna--6 siung bawang merah--4 siun...,"Haluskan bumbu (cabai merah, kemiri, bawang, k...",7,/id/resep/4473138-asam-padeh-ikan-tuna


In [ ]:
df.dtypes

Title          object
Ingredients    object
Steps          object
Loves           int64
URL            object
dtype: object

In [ ]:
df.isna().sum()

Title          0
Ingredients    0
Steps          0
Loves          0
URL            0
dtype: int64

In [ ]:
df.drop(columns=["URL", "Loves", "Steps"], inplace=True)
df.dropna(inplace=True)
df.head()

,Title,Ingredients
0,Gurame Saus Padang,Bahan utama:--1 ekor gurame--Bumbu untuk saus:...
1,Ikan Kembung Bakar Teflon,"1/2 kg ikan kembung sate, bersihkan--1 buah je..."
2,Mujaer asam pedas manis,1/2 kg ikan mujaer (stok gurame habis)--2 buah...
3,Gurame Asam manis,1/2 kg ikan gurame--2 buah wortel (potong sepe...
4,Asam padeh ikan tuna,1/2 kg ikan tuna--6 siung bawang merah--4 siun...


In [ ]:
# Akses 500 raw
df= df.iloc[:500, :]
df

,Title,Ingredients
0,Gurame Saus Padang,Bahan utama:--1 ekor gurame--Bumbu untuk saus:...
1,Ikan Kembung Bakar Teflon,"1/2 kg ikan kembung sate, bersihkan--1 buah je..."
2,Mujaer asam pedas manis,1/2 kg ikan mujaer (stok gurame habis)--2 buah...
3,Gurame Asam manis,1/2 kg ikan gurame--2 buah wortel (potong sepe...
4,Asam padeh ikan tuna,1/2 kg ikan tuna--6 siung bawang merah--4 siun...
...,...,...
495,Woku ikan tongkol,"500 gram ikan tongkol, dipotong-potong--1 send..."
496,Sphagetti saos tuna pedas,"100 gr sphagetti--2 bh bawang putih,rajang hal..."
497,Salmon saos tiram,/+ 2 ons ikan salmon iris tipis--5 sendok saos...
498,Oseng Buncis & Wortel Kombinasi Pentol Tenggiri,3 siung bawang putih--5 siung bawang merah--5 ...


In [ ]:
Ingredients = pd.DataFrame(df.Ingredients)
Ingredients

,Ingredients
0,Bahan utama:--1 ekor gurame--Bumbu untuk saus:...
1,"1/2 kg ikan kembung sate, bersihkan--1 buah je..."
2,1/2 kg ikan mujaer (stok gurame habis)--2 buah...
3,1/2 kg ikan gurame--2 buah wortel (potong sepe...
4,1/2 kg ikan tuna--6 siung bawang merah--4 siun...
...,...
495,"500 gram ikan tongkol, dipotong-potong--1 send..."
496,"100 gr sphagetti--2 bh bawang putih,rajang hal..."
497,/+ 2 ons ikan salmon iris tipis--5 sendok saos...
498,3 siung bawang putih--5 siung bawang merah--5 ...


In [ ]:
Title = pd.DataFrame(df.Title)
Title

,Title
0,Gurame Saus Padang
1,Ikan Kembung Bakar Teflon
2,Mujaer asam pedas manis
3,Gurame Asam manis
4,Asam padeh ikan tuna
...,...
495,Woku ikan tongkol
496,Sphagetti saos tuna pedas
497,Salmon saos tiram
498,Oseng Buncis & Wortel Kombinasi Pentol Tenggiri


In [ ]:
# Function to clean text
def clean_ingredients(text):
    text = re.sub(r'\s*,\s*|\([^)]*\)|,\s*$', '', str(text))

    text = re.sub(r'\bsecukupnya\b', '', text, flags=re.IGNORECASE)
    text = re.sub(r'\b- sesuaikan sesuaikan\b', '', text, flags=re.IGNORECASE)
    text = re.sub(r'\s*,\s*', ',', str(text))
    emoticons = [':)', ':(', ':-)', ':-(', '<3','❤','🍅','🌶','🍗','🔅','👇','🐔','🍛','🌰','📓','🌱']
    for emoticon in emoticons:
        text = text.replace(emoticon, '')
    text = text.replace(':', '')

    # Eliminating some stopwords
    stop_words = set(stopwords.words('indonesian'))
    stop_words.update(['secukupnya','bumbu','bumbu halus','kating',' - sesuaikan sesuaikan','bahan kuah'])
    words = text.split()
    filtered_words = [word.lower() for word in words if word.lower() not in stop_words]
    cleaned_text = ' '.join(filtered_words)

    return cleaned_text

In [ ]:
# Function Delete emoticons in text
def remove_emoji(text):

    emoji_pattern = re.compile("["
                               u"\U0001F600-\U0001F64F"  # emoticons
                               u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                               u"\U0001F680-\U0001F6FF"  # transport & map symbols
                               u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                               u"\U00002702-\U000027B0"
                               u"\U000024C2-\U0001F251"
                               "]+", flags=re.UNICODE)


    no_emoji_list = emoji_pattern.sub(r'', text)

    return no_emoji_list

In [ ]:
# Function Delete a link in a text
def remove_links(text):
    link_pattern = re.compile(r'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\\(\\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+')
    no_link_list = link_pattern.sub(r'', text)

    return no_link_list

In [ ]:
df_ingridients = pd.DataFrame(Ingredients)

# Clearing data in the 'Ingredients' column
df_ingridients['Ingredients'] = df_ingridients['Ingredients'].apply(clean_ingredients)
df_ingridients['Ingredients'] = df_ingridients['Ingredients'].apply(remove_links)
df_ingridients['Ingredients'] = df_ingridients['Ingredients'].apply(remove_emoji)
df_ingridients['Ingredients'] = df_ingridients['Ingredients'].str.split('--')

df_ingridients

,Ingredients
0,"[bahan utama, 1 ekor gurame, bumbu saus, 4 siu..."
1,"[1/2 kg ikan kembung satebersihkan, 1 buah jer..."
2,"[1/2 kg ikan mujaer , 2 buah wortel potong kor..."
3,"[1/2 kg ikan gurame, 2 buah wortel , bumbu, 5 ..."
4,"[1/2 kg ikan tuna, 6 siung bawang merah, 4 siu..."
...,...
495,"[500 gram ikan tongkoldipotong-potong, 1 sendo..."
496,"[100 gr sphagetti, 2 bh bawang putihrajang hal..."
497,"[/+ 2 ons ikan salmon iris tipis, 5 sendok sao..."
498,"[3 siung bawang putih, 5 siung bawang merah, 5..."


In [ ]:
df_title = pd.DataFrame(Title)

# Clearing data in the 'Title' column
df_title['Title'] = df_title['Title'].apply(clean_ingredients)
df_title['Title'] = df_title['Title'].apply(remove_links)
df_title['Title'] = df_title['Title'].apply(remove_emoji)

df_title

,Title
0,gurame saus padang
1,ikan kembung bakar teflon
2,mujaer asam pedas manis
3,gurame asam manis
4,asam padeh ikan tuna
...,...
495,woku ikan tongkol
496,sphagetti saos tuna pedas
497,salmon saos tiram
498,oseng buncis & wortel kombinasi pentol tenggiri


In [ ]:
#Tambah kolom df_ingridients yang sudah bersih ke dalam data set
df = pd.DataFrame(df)
df['New Ingridients'] = df_ingridients
df['New Title'] = df_title
df

,Title,Ingredients,New Ingridients,New Title
0,Gurame Saus Padang,Bahan utama:--1 ekor gurame--Bumbu untuk saus:...,"[bahan utama, 1 ekor gurame, bumbu saus, 4 siu...",gurame saus padang
1,Ikan Kembung Bakar Teflon,"1/2 kg ikan kembung sate, bersihkan--1 buah je...","[1/2 kg ikan kembung satebersihkan, 1 buah jer...",ikan kembung bakar teflon
2,Mujaer asam pedas manis,1/2 kg ikan mujaer (stok gurame habis)--2 buah...,"[1/2 kg ikan mujaer , 2 buah wortel potong kor...",mujaer asam pedas manis
3,Gurame Asam manis,1/2 kg ikan gurame--2 buah wortel (potong sepe...,"[1/2 kg ikan gurame, 2 buah wortel , bumbu, 5 ...",gurame asam manis
4,Asam padeh ikan tuna,1/2 kg ikan tuna--6 siung bawang merah--4 siun...,"[1/2 kg ikan tuna, 6 siung bawang merah, 4 siu...",asam padeh ikan tuna
...,...,...,...,...
495,Woku ikan tongkol,"500 gram ikan tongkol, dipotong-potong--1 send...","[500 gram ikan tongkoldipotong-potong, 1 sendo...",woku ikan tongkol
496,Sphagetti saos tuna pedas,"100 gr sphagetti--2 bh bawang putih,rajang hal...","[100 gr sphagetti, 2 bh bawang putihrajang hal...",sphagetti saos tuna pedas
497,Salmon saos tiram,/+ 2 ons ikan salmon iris tipis--5 sendok saos...,"[/+ 2 ons ikan salmon iris tipis, 5 sendok sao...",salmon saos tiram
498,Oseng Buncis & Wortel Kombinasi Pentol Tenggiri,3 siung bawang putih--5 siung bawang merah--5 ...,"[3 siung bawang putih, 5 siung bawang merah, 5...",oseng buncis & wortel kombinasi pentol tenggiri


In [ ]:
df = pd.DataFrame(df)
df_cleaned = df.dropna()

In [ ]:
dataset_new = pd.DataFrame(df_title)
dataset_new['New Ingridients'] = df_ingridients
dataset_new

,Title,New Ingridients
0,gurame saus padang,"[bahan utama, 1 ekor gurame, bumbu saus, 4 siu..."
1,ikan kembung bakar teflon,"[1/2 kg ikan kembung satebersihkan, 1 buah jer..."
2,mujaer asam pedas manis,"[1/2 kg ikan mujaer , 2 buah wortel potong kor..."
3,gurame asam manis,"[1/2 kg ikan gurame, 2 buah wortel , bumbu, 5 ..."
4,asam padeh ikan tuna,"[1/2 kg ikan tuna, 6 siung bawang merah, 4 siu..."
...,...,...
495,woku ikan tongkol,"[500 gram ikan tongkoldipotong-potong, 1 sendo..."
496,sphagetti saos tuna pedas,"[100 gr sphagetti, 2 bh bawang putihrajang hal..."
497,salmon saos tiram,"[/+ 2 ons ikan salmon iris tipis, 5 sendok sao..."
498,oseng buncis & wortel kombinasi pentol tenggiri,"[3 siung bawang putih, 5 siung bawang merah, 5..."


In [ ]:
bahan = df['New Ingridients']
judul = df['Title']

In [ ]:
# Creating a word replacement dictionary
kamus_penggantian = {
    r'\bb\b': 'bawang',
    r'\bbh\b': 'buah',
}
# Function to replace words with regex
def ganti_kata(kalimat, kamus):
    for pola, ganti_dengan in kamus.items():
        kalimat = re.sub(pola, ganti_dengan, kalimat)
    return kalimat


In [ ]:
#Function to clean text
def clean_ingredients(text):
    text = re.sub(r'\d+|\s+|\-|\/', ' ', str(text))
    text =re.sub(r'''\d+|\s+|\-|\/|kg|gr|lembar|bahan|serbaguna|siung|ruas|jari|bungkus|buah|instan uk + campur tepung terigu|jdyg bsah. beli pasaraku beli rb
    cm|ekr|ne i cuci bersih ptng air jeruk nipis cuci lagi|ekor|ml|sdm|adonan basah|sdt|iris|sck|menggoreng|proteiin tinggi|bh|air merebus mie|kating|cuci bersih''', ' ', str(text))
    text = re.sub(r"['\[\]]", '', text)
    stop_words = set(stopwords.words('indonesian'))
    stop_words.update(['secukupnya','bumbu','bumbu halus','kating',' - sesuaikan sesuaikan','bahan kuah'])
    words = text.split()
    filtered_words = [word.lower() for word in words if word.lower() not in stop_words]
    cleaned_text = ' '.join(filtered_words)

    return cleaned_text

In [ ]:
# Cleaning data
bahan = bahan.apply(clean_ingredients)
judul = judul.apply(clean_ingredients)


In [ ]:
bahan = pd.DataFrame(bahan)
bahan

,New Ingridients
0,"utama, gurame, saus, bawang putih , bawang mer..."
1,"ikan kembung satebersihkan, jeruk sate jeruk k..."
2,"ikan mujaer , wortel potong korek api, bawang ..."
3,"ikan gurame, wortel , bumbu, bawang merah, baw..."
4,"ikan tuna, bawang merah, bawang putih, kemiri,..."
...,...
495,"am ikan tongkoldipotong potong, sendok teh air..."
496,"sphagetti, bawang putihrajang halus, bawang bo..."
497,"+ ons ikan salmon tipis, sendok saos tiram, ba..."
498,"bawang putih, bawang merah, biji cabe rawit, b..."


In [ ]:
judul=pd.DataFrame(judul)
judul

,Title
0,gurame saus padang
1,ikan kembung bakar teflon
2,mujaer asam pedas manis
3,gurame asam manis
4,asam padeh ikan tuna
...,...
495,woku ikan tongkol
496,sphagetti saos tuna pedas
497,salmon saos tiram
498,oseng buncis & wortel kombinasi pentol tenggiri


In [ ]:
dataset_new = pd.DataFrame(judul)
dataset_new['Ingridients'] = bahan
dataset_new

,Title,Ingridients
0,gurame saus padang,"utama, gurame, saus, bawang putih , bawang mer..."
1,ikan kembung bakar teflon,"ikan kembung satebersihkan, jeruk sate jeruk k..."
2,mujaer asam pedas manis,"ikan mujaer , wortel potong korek api, bawang ..."
3,gurame asam manis,"ikan gurame, wortel , bumbu, bawang merah, baw..."
4,asam padeh ikan tuna,"ikan tuna, bawang merah, bawang putih, kemiri,..."
...,...,...
495,woku ikan tongkol,"am ikan tongkoldipotong potong, sendok teh air..."
496,sphagetti saos tuna pedas,"sphagetti, bawang putihrajang halus, bawang bo..."
497,salmon saos tiram,"+ ons ikan salmon tipis, sendok saos tiram, ba..."
498,oseng buncis & wortel kombinasi pentol tenggiri,"bawang putih, bawang merah, biji cabe rawit, b..."


In [ ]:
df = pd.read_csv("INIIKAN.csv")

Sentences-Transfoorm



In [ ]:
pip install sentence-transformers


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 946.9 kB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 9.0 MB/s eta 0:00:00
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125923 sha256=d78fa68913bc9f40505d30fa99a3dc28cb640b2ad59705e5c286f5fbb269fec1
  Stored in directory: /root/.cache/pip/wheels/62/f2/10/1e606fd5f02395388f74e7462910fe851042f97238cbbd902f
Successfully built sentence-transformers


In [ ]:
from sentence_transformers import SentenceTransformer, util
import pandas as pd

In [ ]:
df = pd.DataFrame(df)

In [ ]:
def recommend_recipe(selected_ingredients, df, model_path="distilbert-base-nli-stsb-mean-tokens"):
    # Load model of Hugging Face Transformers
    model = SentenceTransformer(model_path)

    # Merge selected ingredients into one text
    selected_text = ' '.join(selected_ingredients)

    # Encode input text into vectors
    selected_vector = model.encode(selected_text, convert_to_tensor=True)

    # Encode the entire dataset
    recipe_vectors = model.encode(df['Ingridients'].tolist(), convert_to_tensor=True)

    # Calculate cosine similarity between input vector and dataset vector
    similarities = util.pytorch_cos_sim(selected_vector, recipe_vectors)[0]

    # Take the recipe index with the highest similarity
    recommended_indices = similarities.argsort(descending=True)[:5]

    # Grab the recommended recipe title
    recommended_recipes = df['Title'].iloc[recommended_indices]

    return recommended_recipes

In [ ]:
selected_ingredients = ['ikan','gurame']
recommendations = recommend_recipe(selected_ingredients, df)
print(recommendations)

481                   bandeng goreng ketumbar
76                 bandeng suwir sambel korek
288    gulai ikan nila tempoyak (asam durian)
236                           ikan mas goreng
354                  lele goreng kriuk simple
Name: Title, dtype: object


MODEL GENSIM


In [ ]:
pip install gensim

In [ ]:
from gensim.models import Doc2Vec
from gensim.models.doc2vec import TaggedDocument
from sklearn.feature_extraction.text import TfidfVectorizer
import pandas as pd

In [ ]:
df = pd.DataFrame(df)

In [ ]:
df

,Title,Ingridients,combined
0,gurame saus padang,"utama, gurame, saus, bawang putih , bawang mer...","gurame saus padang utama, gurame, saus, bawang..."
1,ikan kembung bakar teflon,"ikan kembung satebersihkan, jeruk sate jeruk k...",ikan kembung bakar teflon ikan kembung sateber...
2,mujaer asam pedas manis,"ikan mujaer , wortel potong korek api, bawang ...","mujaer asam pedas manis ikan mujaer , wortel p..."
3,gurame asam manis,"ikan gurame, wortel , bumbu, bawang merah, baw...","gurame asam manis ikan gurame, wortel , bumbu,..."
4,asam padeh ikan tuna,"ikan tuna, bawang merah, bawang putih, kemiri,...","asam padeh ikan tuna ikan tuna, bawang merah, ..."
...,...,...,...
495,woku ikan tongkol,"am ikan tongkoldipotong potong, sendok teh air...",woku ikan tongkol am ikan tongkoldipotong poto...
496,sphagetti saos tuna pedas,"sphagetti, bawang putihrajang halus, bawang bo...","sphagetti saos tuna pedas sphagetti, bawang pu..."
497,salmon saos tiram,"+ ons ikan salmon tipis, sendok saos tiram, ba...","salmon saos tiram + ons ikan salmon tipis, sen..."
498,oseng buncis & wortel kombinasi pentol tenggiri,"bawang putih, bawang merah, biji cabe rawit, b...",oseng buncis & wortel kombinasi pentol tenggir...


In [ ]:
# Tokenization and Doc2Vec model creation
tagged_data = [TaggedDocument(words=doc.split(), tags=[str(i)]) for i, doc in enumerate(df['combined'])]
model = Doc2Vec(vector_size=100, window=5, min_count=1, workers=4, epochs=20)

# Build vocabulary
model.build_vocab(tagged_data)

# Train the model
model.train(tagged_data, total_examples=model.corpus_count, epochs=model.epochs)


In [ ]:
model.save("doc2vec_model.model")

In [ ]:
# Recipe recommendation function
def recommend_recipe(selected_ingredients, df, model_path="doc2vec_model.model"):
    # Load model from file
    model = Doc2Vec.load(model_path)

    # Merge selected ingredients into one text
    selected_text = ' '.join(selected_ingredients)

    # Infer vector for the selected text
    selected_vector = model.infer_vector(selected_text.split())

    # Temukan resep-resep yang paling mirip
    similar_recipes = model.dv.most_similar([selected_vector], topn=5)

    # Grab the recommended recipe title
    recommended_indices = [int(idx) for idx, _ in similar_recipes]
    recommended_recipes = df['Title'].iloc[recommended_indices]

    return recommended_recipes

In [ ]:
selected_ingredients = ['ikan', 'gurame']
recommendations = recommend_recipe(selected_ingredients, df)
print(recommendations)

151                    nugget ikan tuna
287               suwir tongkol kemangi
201           dori fillet bawang goreng
214            ikan nila goreng praktis
157    tuna goreng crispy saus teriyaki
Name: Title, dtype: object


END GENSIM


In [ ]:
# Build tokenizer
tokenizer = Tokenizer()
tokenizer.fit_on_texts(judul)
# Tokenizing and padding
sequences = tokenizer.texts_to_sequences(judul)
padded_sequences = pad_sequences(sequences, maxlen=3, padding='post', truncating='post')
# New title reconstruction
judul_baru = []
for sequence in padded_sequences:
    words = [word for word, index in tokenizer.word_index.items() if index in sequence]
    judul_baru.append(' '.join(words))

In [ ]:
# Membangun tokenizer
tokenizer = Tokenizer()
tokenizer.fit_on_texts(bahan)
# Tokenizing and padding
sequences = tokenizer.texts_to_sequences(bahan)
padded_sequences = pad_sequences(sequences, maxlen=100, padding='post', truncating='post')
# New title reconstruction
bahan_baru = []
for sequence in padded_sequences:
    words = [word for word, index in tokenizer.word_index.items() if index in sequence]
    bahan_baru.append(' '.join(words))

In [ ]:
print(df)

                                               Title  \
0                                 gurame saus padang   
1                          ikan kembung bakar teflon   
2                            mujaer asam pedas manis   
3                                  gurame asam manis   
4                               asam padeh ikan tuna   
..                                               ...   
495                                woku ikan tongkol   
496                        sphagetti saos tuna pedas   
497                                salmon saos tiram   
498  oseng buncis & wortel kombinasi pentol tenggiri   
499                  ikan nila madu saos cabai hijau   

                                           Ingridients  \
0    utama, gurame, saus, bawang putih , bawang mer...   
1    ikan kembung satebersihkan, jeruk sate jeruk k...   
2    ikan mujaer , wortel potong korek api, bawang ...   
3    ikan gurame, wortel , bumbu, bawang merah, baw...   
4    ikan tuna, bawang merah, bawang 

In [ ]:
tfidf_vectorizer = TfidfVectorizer(stop_words='english')

In [ ]:
tfidf_matrix = tfidf_vectorizer.fit_transform(df['combined'])

In [ ]:
cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)

In [ ]:
import pickle

# Simpan cosine_sim ke dalam file .pkl
with open('recipe_recommendation.pkl', 'wb') as file:
    pickle.dump(cosine_sim, file)


In [ ]:
def recommend_recipe(selected_ingredients, df, cosine_sim=cosine_sim):
    # Merge selected ingredients into one text
    selected_text = ' '.join(selected_ingredients)

    # Create a copy of the DataFrame
    df_temp = df.copy()

    # Add a 'selected' column with the text of the selected food ingredient
    df_temp['selected'] = selected_text

    # Merge the recipe title with the selected ingredient text
    df_temp['combined_selected'] = df_temp['Title'] + ' ' + df_temp['selected']

    # Calculate TF-IDF matrix for selected texts
    tfidf_matrix_selected = tfidf_vectorizer.transform(df_temp['combined_selected'])

    # Calculate the cosine similarity between the recipe and the selected text
    cosine_sim_selected = linear_kernel(tfidf_matrix_selected, tfidf_matrix)

   # Get the recommended recipe index based on the highest similarity
    recommended_indices = cosine_sim_selected[0].argsort()[:-6:-1]

    # Filter recipe index based on similarity with recipe title
    filtered_indices = [idx for idx in recommended_indices if selected_text.lower() in df['Title'].iloc[idx].lower()]

    # Determine the number of recommendations still needed
    remaining_recommendations = 5 - len(filtered_indices)

   # Get additional indexes if needed
    additional_indices = [idx for idx in recommended_indices if idx not in filtered_indices][:remaining_recommendations]

    # Merge the filtered index with additional indexes
    filtered_indices += additional_indices

    # Grab the recommended recipe title
    recommended_recipes = df['Title'].iloc[filtered_indices]

    return recommended_recipes

In [ ]:
df

,Title,Ingridients,combined
0,gurame saus padang,"utama, gurame, saus, bawang putih , bawang mer...","gurame saus padang utama, gurame, saus, bawang..."
1,ikan kembung bakar teflon,"ikan kembung satebersihkan, jeruk sate jeruk k...",ikan kembung bakar teflon ikan kembung sateber...
2,mujaer asam pedas manis,"ikan mujaer , wortel potong korek api, bawang ...","mujaer asam pedas manis ikan mujaer , wortel p..."
3,gurame asam manis,"ikan gurame, wortel , bumbu, bawang merah, baw...","gurame asam manis ikan gurame, wortel , bumbu,..."
4,asam padeh ikan tuna,"ikan tuna, bawang merah, bawang putih, kemiri,...","asam padeh ikan tuna ikan tuna, bawang merah, ..."
...,...,...,...
495,woku ikan tongkol,"am ikan tongkoldipotong potong, sendok teh air...",woku ikan tongkol am ikan tongkoldipotong poto...
496,sphagetti saos tuna pedas,"sphagetti, bawang putihrajang halus, bawang bo...","sphagetti saos tuna pedas sphagetti, bawang pu..."
497,salmon saos tiram,"+ ons ikan salmon tipis, sendok saos tiram, ba...","salmon saos tiram + ons ikan salmon tipis, sen..."
498,oseng buncis & wortel kombinasi pentol tenggiri,"bawang putih, bawang merah, biji cabe rawit, b...",oseng buncis & wortel kombinasi pentol tenggir...


In [ ]:
selected_ingredients = ["ikan nila","tomat","nila"]
recommendations = recommend_recipe(selected_ingredients, df)
print(recommendations)

214               ikan nila goreng praktis
241                 sambel tomat ikan nila
293                ikan nila goreng balado
473    ikan nila acar atw pesmol ikan nila
486                 nila goreng saus tiram
Name: Title, dtype: object


In [ ]:
df = pd.DataFrame(dataset_new)

In [ ]:
df

,Title,Ingridients,combined
0,gurame saus padang,"utama, gurame, saus, bawang putih , bawang mer...","gurame saus padang utama, gurame, saus, bawang..."
1,ikan kembung bakar teflon,"ikan kembung satebersihkan, jeruk sate jeruk k...",ikan kembung bakar teflon ikan kembung sateber...
2,mujaer asam pedas manis,"ikan mujaer , wortel potong korek api, bawang ...","mujaer asam pedas manis ikan mujaer , wortel p..."
3,gurame asam manis,"ikan gurame, wortel , bumbu, bawang merah, baw...","gurame asam manis ikan gurame, wortel , bumbu,..."
4,asam padeh ikan tuna,"ikan tuna, bawang merah, bawang putih, kemiri,...","asam padeh ikan tuna ikan tuna, bawang merah, ..."
...,...,...,...
495,woku ikan tongkol,"am ikan tongkoldipotong potong, sendok teh air...",woku ikan tongkol am ikan tongkoldipotong poto...
496,sphagetti saos tuna pedas,"sphagetti, bawang putihrajang halus, bawang bo...","sphagetti saos tuna pedas sphagetti, bawang pu..."
497,salmon saos tiram,"+ ons ikan salmon tipis, sendok saos tiram, ba...","salmon saos tiram + ons ikan salmon tipis, sen..."
498,oseng buncis & wortel kombinasi pentol tenggiri,"bawang putih, bawang merah, biji cabe rawit, b...",oseng buncis & wortel kombinasi pentol tenggir...


In [ ]:
pip install scikit-surprise

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 772.0/772.0 kB 9.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.3-cp310-cp310-linux_x86_64.whl size=3163762 sha256=6943cbdff07c7a782722afbe9648a8ef86299b0cec08e57f7af3f1b8d19cfcf5
  Stored in directory: /root/.cache/pip/wheels/a5/ca/a8/4e28def53797fdc4363ca4af740db15a9c2f1595ebc51fb445
Successfully built scikit-surprise


In [ ]:
from surprise import Dataset, Reader, SVD
from surprise.model_selection import train_test_split
from surprise.dump import dump

In [ ]:
reader = Reader(rating_scale=(1, 5))

In [ ]:
df['Rating'] = 5

In [ ]:
df

,Title,Ingridients,combined,Rating
0,gurame saus padang,"utama, gurame, saus, bawang putih , bawang mer...","gurame saus padang utama, gurame, saus, bawang...",5
1,ikan kembung bakar teflon,"ikan kembung satebersihkan, jeruk sate jeruk k...",ikan kembung bakar teflon ikan kembung sateber...,5
2,mujaer asam pedas manis,"ikan mujaer , wortel potong korek api, bawang ...","mujaer asam pedas manis ikan mujaer , wortel p...",5
3,gurame asam manis,"ikan gurame, wortel , bumbu, bawang merah, baw...","gurame asam manis ikan gurame, wortel , bumbu,...",5
4,asam padeh ikan tuna,"ikan tuna, bawang merah, bawang putih, kemiri,...","asam padeh ikan tuna ikan tuna, bawang merah, ...",5
...,...,...,...,...
495,woku ikan tongkol,"am ikan tongkoldipotong potong, sendok teh air...",woku ikan tongkol am ikan tongkoldipotong poto...,5
496,sphagetti saos tuna pedas,"sphagetti, bawang putihrajang halus, bawang bo...","sphagetti saos tuna pedas sphagetti, bawang pu...",5
497,salmon saos tiram,"+ ons ikan salmon tipis, sendok saos tiram, ba...","salmon saos tiram + ons ikan salmon tipis, sen...",5
498,oseng buncis & wortel kombinasi pentol tenggiri,"bawang putih, bawang merah, biji cabe rawit, b...",oseng buncis & wortel kombinasi pentol tenggir...,5


In [ ]:
data_surprise = Dataset.load_from_df(df[['Title', 'Ingridients', 'Rating']], reader)

In [ ]:
trainset, testset = train_test_split(data_surprise, test_size=0.2)

In [ ]:
model = SVD()
model.fit(trainset)

In [ ]:
model_filename = 'recipe_recommendation_model.h5'
dump(model_filename, algo=model)

print(f"Model saved as {model_filename}")

Model saved as recipe_recommendation_model.h5


DONT RUNNNNNNN

In [ ]:
data_surprise = Dataset.load_from_df(df[['Title', 'Ingridients', 'Rating']], reader)

In [ ]:
import pickle
with open('tokenizer.pkl', 'wb') as tokenizer_file:
    pickle.dump(tokenizer, tokenizer_file)

with open('tfidf_vectorizer.pkl', 'wb') as tfidf_vectorizer_file:
    pickle.dump(tfidf_vectorizer, tfidf_vectorizer_file)

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

In [ ]:
model = Sequential()
model.add(Dense(64, input_shape=(100,), activation='relu'))
model.add(Dense(1, activation='sigmoid'))
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [ ]:
model.save('recommendation_model.h5')